# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("Output/city_data.csv")
city_weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Temperature (F),Temp Max,Humidity (%),Cloudiness (%),Windspeed (mph)
0,0,Jennings,US,38.72,-90.26,1600208260,78.69,81.00,44,75,5.82
1,1,Bluff,NZ,-46.60,168.33,1600208086,48.99,48.99,79,100,14.00
2,2,Kapaa,US,22.08,-159.32,1600208097,84.79,86.00,58,20,10.29
3,3,Lebu,CL,-37.62,-73.65,1600208260,50.95,50.95,96,100,6.06
4,4,Grand-Santi,GF,4.25,-54.38,1600208260,75.85,75.85,90,81,0.54
...,...,...,...,...,...,...,...,...,...,...,...
579,579,Selaphum,TH,16.03,103.94,1600208316,77.11,77.11,85,77,2.42
580,580,Turochak,RU,52.26,87.12,1600208316,47.59,47.59,80,82,3.53
581,581,Tingi,TZ,-11.30,35.03,1600208316,68.38,68.38,52,31,6.89
582,582,Nelson Bay,AU,-32.72,152.15,1600207872,59.63,60.01,93,20,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["Latitude", "Longitude"]].astype(float)

# Create a poverty Heatmap layer
humidity_fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=city_weather_df['Humidity (%)'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

humidity_fig.add_layer(heat_layer)
humidity_fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
print(f' The figure shows the heat map of the relative humidity percent in each city')

 The figure shows the heat map of the relative humidity percent in each city


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_cities_df = city_weather_df.loc[(city_weather_df["Temp Max"] > 70) & (city_weather_df["Temp Max"] < 80) & (city_weather_df["Cloudiness (%)"] == 0) & (city_weather_df["Humidity (%)"] < 50), :]
ideal_cities_df = ideal_cities_df.dropna(how='any')
del ideal_cities_df['Unnamed: 0']
hotel_df=ideal_cities_df.reset_index(drop=True)
hotel_df

,City,Country,Latitude,Longitude,Date,Temperature (F),Temp Max,Humidity (%),Cloudiness (%),Windspeed (mph)
0,Piracanjuba,BR,-17.30,-49.02,1600207603,79.81,79.81,22,0,10.22
1,Areni,AM,39.72,45.18,1600208309,77.00,77.00,44,0,11.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#add column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Date,Temperature (F),Temp Max,Humidity (%),Cloudiness (%),Windspeed (mph),Hotel Name
0,Piracanjuba,BR,-17.30,-49.02,1600207603,79.81,79.81,22,0,10.22,
1,Areni,AM,39.72,45.18,1600208309,77.00,77.00,44,0,11.41,


In [17]:
#
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get latitude & longitude
    lat = row["Latitude"]
    long = row["Longitude"]

    #loop through parameters
    params["location"] = f"{lat},{long}"
    #loop through this url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make API request
    print(f"Retrieving Results: {row['City']}.")
    response = requests.get(base_url, params=params).json()
   
    
    #results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results: Piracanjuba.
Closest hotel is Hotel das Rosas em Piracanjuba.
Retrieving Results: Areni.
Closest hotel is Park-Hotel Khoren's Lake.


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
fig = gmaps.figure()
markers = gmaps.marker_layer(locations)
# Add marker layer ontop of heat map
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 0, 150, 0)',
                                stroke_color='rgba(0, 0, 150, 0.0)', 
                                scale=2,
                                info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [10]:
print(f' The figure is a map of the locations of the hotels with the most ideal weather: between 75-85F, 0% cloudiness, and less than 50% humidity')

 The figure is a map of the locations of the hotels with the most ideal weather: between 75-85F, 0% cloudiness, and less than 50% humidity
